# **To connect Azure datalake to databricks.**

### **Mehtods of Azure data lakes** 

> ADLS integration with data bricks
> 
> - Using a service principal
> - using Azure active directory credentials knows as a credential passthrough
> - Using ADLS access key directly
> - Creating mount found using ADLS access key

 ### **Follow and execute these code in your Databricks platform.**

### **Method 1: Using ADLS access key directly**

For this we have two configure properties.

In [ ]:
spark.conf.set (
"fs.azure.account.key.<storage account>.dfs.core.window.net",
"<access key>"
)

*   \<storage account\> - 
    *   for this you need to create a storage account
        *   Provide a unique name for your Storage account. The name must be unique across Azure.
*   \<access keys\>
    *   Go to storage account and on left you can access keys in security + networking section.

**To list out the files in the contianer**
*   Container is like folder in azure data storage.

In [ ]:
dbutils.fs.ls("abfss://<container_name>@<storage_account_name>.dfs.core.window.net/")

**To read the file present in the container**

In [ ]:
# set the data lake file location
file_location = "abfss://<container_name>@<storage_account_name>.dfs.core.window.net/"

# under this folder we have csv file which we will convert to dataframe.

# to create data frame from file
# the file is fetched/extracted from azur data storage into databric and we create dataframe.
df= spark.read.format("csv").option("inferschema",True).option("header",True).option("delimiter",",").load(file_location)

# to display the dataframe
df.show()

### **Method 2: Creating Mount Point using ADLS Access Key**

Once mount point is created than databricks can consider Azur data lake storage as local file system even if its not local file system.

It will read and write as if it is local file system for databricks.

**For this we need to set some parameters**
*   source
    *   Go to storage account, click on  go to resources.
    *   On left in setting find endpoint setting and click on that.
    *   Copy the Primary endpoint Blob services. 
    *   copy the part after https://
    *   source = "wasbs://<container_name>@<copied_part>"
    *   wasbs - part of syntax to connect using blob services.
*   mount_point
    *    It's essentially the directory path on the Spark cluster where you want to access the files from the mounted storage.
    *    The mount_point is set to "/mnt/". This means that the content of the storage container specified in the source parameter will be mounted at the "/mnt/" directory on the Spark cluster.
    *   For example, if you set mount_point="/my_mount_point/", the content of the storage container will be accessible under the "/my_mount_point/" directory on the Spark cluster.

In [ ]:
dbutils.fs.mount(
source = "wasbs://<container_name>@<storage_account_name>.blob.core.windows.net",
mount_point = "/mnt/adls_test"
extra_configs={"fs.azure.account.key.<storage_account_name>.blob.core.windows.net":"<acess key>"}

)

**Now Mount point is created successfully**
    *   Now we don't have to mention the whole path of azure data storage to fect the data we can get that from mount point folder also.


In [ ]:
# To list out the files present in the path mentioned.
dbutils.fs.ls("/mnt/adls_test")

**To list out the mount point we created**

In [ ]:
dbutils.fs.mounts()

**To unmount the point**
*   dbutils.fs.unmount("point")
*   point - ths path or point which you want to mount.

In [ ]:
dbutils.fs.unmount("/mnt/adls_test")